In [3]:
import cobra

In [4]:
met = cobra.io.read_sbml_model("./data/iMcBath_rewritten.xml")

In [5]:
met.optimize()

,fluxes,reduced_costs
2OH3K5MPPISO,0.000000,0.000000
34DHOXPEGOX,0.000000,-0.000000
3HAD100,-0.005615,-0.000000
3HAD120,0.005615,0.000000
3HAD140,0.003835,0.000000
...,...,...
DM_oaa_c,0.000000,-0.096448
DM_akg_c,0.000000,-0.154316
DM_3mob_c,0.000000,-0.212185
SK_h2_c,0.000000,-0.019290


In [6]:
met.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.01244,0,0.00%
ch4_e,EX_ch4_e,18.46,1,100.00%
cl_e,EX_cl_e,0.03817,0,0.00%
cobalt2_e,EX_cobalt2_e,1.027E-05,0,0.00%
cu2_e,EX_cu2_e,0.000269,0,0.00%
fe2_e,EX_fe2_e,0.00109,0,0.00%
h_e,EX_h_e,1.458,0,0.00%
k_e,EX_k_e,0.03142,0,0.00%
mg2_e,EX_mg2_e,0.02198,0,0.00%
mn2_e,EX_mn2_e,3.205E-06,0,0.00%


It is clearly seen that in M. capsulatus the uptake of methane outstands among all the other uptakes. Thus, now we will check those reactions where methane is involved. Besides, we will also check the methanol-involved reactions as it is here where our main interest is placed.

IMPORTANT!! Consider using the query both with initial letter as capital letter and not capital!

In [7]:
met.reactions.query("Methane", "name")

[<Reaction PMMODCipp at 0x7f69d12ef2e0>,
 <Reaction PMMOipp at 0x7f69d12efaf0>,
 <Reaction SMMOi at 0x7f69d1eecb50>]

In [8]:
met.reactions.EX_ch4_e

Reaction identifier,EX_ch4_e
Name,methane exchange
Memory address,0x07f69d15d0340
Stoichiometry,ch4_e <-- methane <--
GPR,
Lower bound,-18.46
Upper bound,0.0


Question: Why is it that the lb is placed to -18.46? Also interesting that this is exactly the optimal value obtained when optimizing.

Obtained from the paper for the model:
The uptake rate for methane in iMcBath was constrained to 18.46 mmol gDW-1 h-1, a value adopted from de la Torre et al. (2015) due to the lack of a specific measurement for M. capsulatus (Bath).

Should we take this limit value from the experimental killing curves from wet lab?

In [9]:
met.reactions.CH4tex

Reaction identifier,CH4tex
Name,methane transport via diffusion (extracellular to periplasm)
Memory address,0x07f69d16f2d30
Stoichiometry,ch4_e <=> ch4_p methane <=> methane
GPR,
Lower bound,-1000.0
Upper bound,1000.0


Question: Why do we consider the diffusion reaction without any limit? What is the relation between the methane obtained from the exchange reaction level and the diffusion? Because if we consider the diffusion with no limit and the is this periplasmatic methane that is taking further down on the reactions, where does the 18.46 limit apply?

In [27]:
print(met.reactions.SMMOi)
print(met.reactions.PMMOipp)
print(met.reactions.PMMODCipp)

SMMOi: ch4_c + h_c + nadh_c + o2_c --> h2o_c + meoh_c + nad_c
PMMOipp: ch4_p + o2_p + q8h2_im --> h2o_p + meoh_p + q8_im
PMMODCipp: ch4_p + 2.0 focytcc555_p + 2.0 h_p + o2_p --> 2.0 ficytcc555_p + h2o_p + meoh_p


For our interest in K. Phaffi:

In [28]:
met.reactions.PMMOipp

Reaction identifier,PMMOipp
Name,particulate Methane Monooxygenase
Memory address,0x010d90c72710
Stoichiometry,ch4_p + o2_p + q8h2_im --> h2o_p + meoh_p + q8_im methane + dioxygen + Ubiquinol-8 --> H2O + methanol + Ubiquinone-8
GPR,( ( MCA1796 and MCA1797 and MCA1798 ) or ( MCA2853 and MCA2854 and MCA2855 ) ) and MCA0295
Lower bound,0.0
Upper bound,0.0


In [29]:
met.reactions.PMMODCipp

Reaction identifier,PMMODCipp
Name,particulate Methane Monooxygenase - Direct Coupling
Memory address,0x010d90c72208
Stoichiometry,ch4_p + 2.0 focytcc555_p + 2.0 h_p + o2_p --> 2.0 ficytcc555_p + h2o_p + meoh_p methane + 2.0 ferrocytochrome c + 2.0 H(+) + dioxygen --> 2.0 ferricytochrome c + H2O + methanol
GPR,( ( MCA1796 and MCA1797 and MCA1798 ) or ( MCA2853 and MCA2854 and MCA2855 ) ) and MCA0295
Lower bound,0.0
Upper bound,0.0


Question: Difference between both? Any reason why both are constrained not to work?

Now we come to the methanol reactions:

In [16]:
met.reactions.EX_meoh_e

Reaction identifier,EX_meoh_e
Name,methanol exchange
Memory address,0x0185c3e44978
Stoichiometry,meoh_e --> methanol -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [6]:
met.reactions.MEOHtex

Reaction identifier,MEOHtex
Name,methanol transport via diffusion (extracellular to periplasm)
Memory address,0x010d90ab9ef0
Stoichiometry,meoh_e <=> meoh_p methanol <=> methanol
GPR,
Lower bound,-1000.0
Upper bound,1000.0


This following reaction is the one that we might need to introduce to out K. phaffii model as it is the one getting from methanol to formaldehyde, starting the MUT pathway?

In [17]:
met.reactions.MEOHDHipp

Reaction identifier,MEOHDHipp
Name,methanol dehydrogenase (cytochrome c)
Memory address,0x0185c405f588
Stoichiometry,2.0 ficytcc555_p + meoh_p --> fald_p + 2.0 focytcc555_p + 2.0 h_p 2.0 ferricytochrome c + methanol --> formaldehyde + 2.0 ferrocytochrome c + 2.0 H(+)
GPR,( ( MCA0299 and MCA0300 and MCA1528 and MCA1529 and MCA1530 and MCA1525 and MCA1527 ) or (...
Lower bound,0.0
Upper bound,1000.0


In [22]:
met.reactions.MEOHtrpp

Reaction identifier,MEOHtrpp
Name,Methanol reversible transport via diffusion (periplasm)
Memory address,0x010d90ab9cc0
Stoichiometry,meoh_p <=> meoh_c methanol <=> methanol
GPR,
Lower bound,-1000.0
Upper bound,1000.0


Also, easily obtained by:

In [17]:
met.metabolites.ch4_c.summary()

Percent,Flux,Reaction,Definition
100.00%,18.46,CH4trpp,ch4_p <=> ch4_c
Percent,Flux,Reaction,Definition
100.00%,-18.46,SMMOi,ch4_c + h_c + nadh_c + o2_c --> h2o_c + meoh_c + nad_c


Notice that in this model, one can refer to extracellular species by adding e, periplasmatic by adding p and intracellular by adding c (cytosol). And by checking the file im refers to inner membrane. This will depend on the model, tends to be specified somewhere on the sbml file under compartments.

In [16]:
met.metabolites.meoh_c.summary()

Percent,Flux,Reaction,Definition
100.00%,18.46,SMMOi,ch4_c + h_c + nadh_c + o2_c --> h2o_c + meoh_c + nad_c
Percent,Flux,Reaction,Definition
100.00%,-18.46,MEOHtrpp,meoh_p <=> meoh_c


In [30]:
met.medium

{'EX_ca2_e': 1000.0,
 'EX_cd2_e': 1000.0,
 'EX_ch4_e': 18.46,
 'EX_cl_e': 1000.0,
 'EX_co_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_no3_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [18]:
met.reactions.query("Amino","name")

[<Reaction APCS at 0x7f69d1719c10>, <Reaction FOMETRi at 0x7f69d1598fd0>]

In [20]:
met.reactions.FOMETRi

Reaction identifier,FOMETRi
Name,Aminomethyltransferase
Memory address,0x07f69d1598fd0
Stoichiometry,"h2o_c + methf_c --> 5fthf_c + h_c H2O + (6R)-5,10-methenyltetrahydrofolate --> 5-Formyltetrahydrofolate + H(+)"
GPR,MCA0350
Lower bound,0.0
Upper bound,1000.0


In [12]:
# now we will introduce the heterologous proteins, pMMO and leghemoglobin, with reactions for dna replication,
# transcription and translation
# as the sequences are long, we calculate the stoichiometry with a script which is on the github repo and
# based on the logic behind introduction of heterologous protein production in the paper:
# Irani ZA, Kerkhoven EJ, Shojaosadati SA, Nielsen J (2016) Genome-scale metabolic model of Pichia pastoris with native and humanized glycosylation of recombinant proteins. Biotechnol Bioeng 113:961–969. doi: 10.1002/bit.25863.

# make new reactions



hemo_DNA_reaction = cobra.Reaction(
            'hemo_DNA',
            name = 'hemoglobin DNA replication',
            lower_bound = 0.0,
            upper_bound = 1000.0
        )

hemo_RNA_reaction = cobra.Reaction(
            'hemo_RNA',
            name = 'hemoglobin transcription',
            lower_bound = 0.0,
            upper_bound = 1000.0
        )

hemo_AA_reaction = cobra.Reaction(
            'hemo_protein',
            name = 'hemoglobin translation',
            lower_bound = 0.0,
            upper_bound = 1000.0
)


# add the metabolites to be produced by these reactions


Hemo_DNA = cobra.Metabolite(
    'Hemo_DNA',
    formula='',
    name='Hemo_DNA',
    compartment='C_c')

Hemo_RNA = cobra.Metabolite(
    'Hemo_RNA',
    formula='',
    name='Hemo_RNA',
    compartment='C_c')

Hemo_AA = cobra.Metabolite(
    'Hemo_AA',
    formula='',
    name='Leghemoglobin',
    compartment='C_c')


met.add_metabolites([Hemo_DNA,
                       Hemo_RNA,
                       Hemo_AA])


# add involved metabolites and stoichiometry



hemo_DNA_reaction.add_metabolites(
    {
        ### deoxynucleotides
        # A
        pheast.metabolites.m404: -1.0424,
        # T
        pheast.metabolites.m437: -1.0424,
        # C
        pheast.metabolites.m431: -0.57636,
        # G
        pheast.metabolites.m389: -0.57636,
        # ATP
        pheast.metabolites.m1: -11.00757,
        # H2O
        pheast.metabolites.m5: -11.00757,
        # Hemo_DNA
        pheast.metabolites.Hemo_DNA: 1.0,
        # ADP
        pheast.metabolites.m3: 11.00757,
        # Phosphate
        pheast.metabolites.m7: 11.00757
    }
)

hemo_RNA_reaction.add_metabolites(
    {
        ### RNA nucleotides
        # A
        pheast.metabolites.m94: -1.2408,
        # U
        pheast.metabolites.m418: -0.80329,
        # C
        pheast.metabolites.m423: -0.6455,
        # G
        pheast.metabolites.m384: -0.43034,
        # ATP
        pheast.metabolites.m1: -7.48783,
        # H2O
        pheast.metabolites.m5: -7.48783,
        # Hemo_RNA
        pheast.metabolites.Hemo_RNA: 1.0,
        # ADP
        pheast.metabolites.m3: 7.48783,
        # Phosphate
        pheast.metabolites.m7: 7.48783
    }
)

hemo_AA_reaction.add_metabolites(
    {
        ### Amino Acids
        # A
        pheast.metabolites.m153: -1.48373,
        # C, no Cystein in leghemoglobin
        # pheast.metabolites.m331:,
        # H
        pheast.metabolites.m490: -0.12902,
        # M
        pheast.metabolites.m343: -0.06451,
        # T
        pheast.metabolites.m305: -0.38706,
        # R
        pheast.metabolites.m158: -0.06451,
        # E
        pheast.metabolites.m154: -0.6451,
        # I
        pheast.metabolites.m239: -0.38706,
        # F
        pheast.metabolites.m272: -0.58059,
        # W
        pheast.metabolites.m280: -0.12902,
        # N
        pheast.metabolites.m161: -0.25804,
        # Q
        pheast.metabolites.m163: -0.32255,
        # L
        pheast.metabolites.m227: -0.83863,
        # P
        pheast.metabolites.m185: -0.32255,
        # Y
        pheast.metabolites.m274: -0.19353,
        # D
        pheast.metabolites.m156: -0.51608,
        # G
        pheast.metabolites.m210: -0.51608,
        # K
        pheast.metabolites.m203: -0.90314,
        # S
        pheast.metabolites.m279: -0.83863,
        # V
        pheast.metabolites.m222: -0.77412,
        # ATP
        pheast.metabolites.m1: -40.22199,
        # H2O
        pheast.metabolites.m5: -40.22199,
        # Leghemoglobin
        pheast.metabolites.Hemo_AA: 1.0,
        # ADP
        pheast.metabolites.m3: 40.22199,
        # Phosphate
        pheast.metabolites.m7: 40.22199
    }
)





# add reactions to pheast

met.add_reactions([hemo_DNA_reaction,
                     hemo_RNA_reaction,
                     hemo_AA_reaction])

# also add transport to extracellular and boundary proteins as well as "Biosynthesis" reactions and the exchange reaction
# this was all done in the paper


hemo_c = cobra.Metabolite(
    'hemo_c',
    formula='',
    name='hemo_c',
    compartment='C_c')

hemo_e = cobra.Metabolite(
    'hemo_e',
    formula='',
    name='hemo_e',
    compartment='C_e')

hemo_b = cobra.Metabolite(
    'hemo_b',
    formula='',
    name='hemo_b',
    compartment='C_b')

met.add_metabolites([pMMO_c,pMMO_e,pMMO_b,
                      hemo_c,hemo_e,hemo_b])

Take_Out_Hemo = cobra.Reaction(
            'Take_Out_Hemo',
            name = 'Take_Out_Hemo',
            lower_bound = 0.0,
            upper_bound = 1000.0
)



extrac_Hemo = cobra.Reaction(
            'c_Hemo_e',
            name = 'extracellular transport Leghemoglobin',
            lower_bound = 0.0,
            upper_bound = 1000.0
)



Hemo_Biosynthesis = cobra.Reaction(
            'Hemo_Biosynthesis',
            name = 'LeghemoglobinBiosynthesis',
            lower_bound = 0.0,
            upper_bound = 1000.0
)

EX_hemo = cobra.Reaction(
            'EX_hemo',
            name = 'hemoglobin exchange reaction',
            lower_bound = -1000.0,
            upper_bound = 1000.0
)


Hemo_Biosynthesis.add_metabolites(
    {
        pheast.metabolites.Hemo_DNA: -2.8e-05,
        
        pheast.metabolites.Hemo_RNA: -0.0029,
        
        pheast.metabolites.Hemo_AA: -0.997,

        pheast.metabolites.hemo_c: 1.0,
    }
)




Take_Out_Hemo.add_metabolites(
    {
        pheast.metabolites.hemo_e: -1.0,   
        
        pheast.metabolites.hemo_b: 1.0
    }
)



extrac_Hemo.add_metabolites(
    {
        pheast.metabolites.hemo_c: -1.0,   
        
        pheast.metabolites.hemo_e: 1.0
    }
)

EX_hemo.add_metabolites(
    {
        pheast.metabolites.hemo_c: -1.0,   
        
        pheast.metabolites.hemo_e: 1.0
    }
)

EX_hemo.add_metabolites(
    {
        pheast.metabolites.pMMO_b: -1.0
    }
)

met.add_reactions([Take_Out_Hemo,
                    extrac_Hemo,
                    Hemo_Biosynthesis,
                    EX_hemo])

NameError: name 'pheast' is not defined